# MLP w TensorFlow



In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

C:\Users\admin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Zad.
Użyj funkcji aktywacji:
```python
f.nn.elu
```

In [3]:
n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [5]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

he_init = tf.contrib.layers.variance_scaling_initializer()

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.elu, kernel_initializer=he_init)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")      

In [6]:
with tf.name_scope("loss"):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss") 

In [7]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)  

In [8]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, tf.argmax(y, 1), 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))   

In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [10]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [11]:
n_epochs = 100
batch_size = 64

In [12]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(mnist.train.num_examples // batch_size):
            print("\r{}%".format(100 * batch_index //  (mnist.train.num_examples // batch_size) ), end="")
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(" ", epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)    
        
    #save_path = saver.save(sess, "./MNIST_MLP/my_model_final_one_hot.ckpt")   

99%  0 Train accuracy: 0.96875 Test accuracy: 0.8989
99%  1 Train accuracy: 0.90625 Test accuracy: 0.912
99%  2 Train accuracy: 0.921875 Test accuracy: 0.919
99%  3 Train accuracy: 0.96875 Test accuracy: 0.9261
99%  4 Train accuracy: 1.0 Test accuracy: 0.9289
99%  5 Train accuracy: 0.953125 Test accuracy: 0.9329
99%  6 Train accuracy: 0.890625 Test accuracy: 0.935
99%  7 Train accuracy: 0.921875 Test accuracy: 0.9396
99%  8 Train accuracy: 0.984375 Test accuracy: 0.9408
99%  9 Train accuracy: 0.984375 Test accuracy: 0.9434
99%  10 Train accuracy: 0.984375 Test accuracy: 0.944
99%  11 Train accuracy: 0.9375 Test accuracy: 0.9478
99%  12 Train accuracy: 0.96875 Test accuracy: 0.9489
99%  13 Train accuracy: 0.96875 Test accuracy: 0.9519
99%  14 Train accuracy: 0.953125 Test accuracy: 0.9532
99%  15 Train accuracy: 0.921875 Test accuracy: 0.9551
99%  16 Train accuracy: 0.984375 Test accuracy: 0.9564
99%  17 Train accuracy: 0.96875 Test accuracy: 0.958
99%  18 Train accuracy: 0.96875 Test a

In [13]:
with tf.Session() as sess:
    saver.restore(sess, "./MNIST_MLP/my_model_final_one_hot.ckpt") 
    X_new_scaled, y_true = mnist.train.next_batch(1) # some new images (scaled from 0 to 1)
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)
print("True label ", y_true)
print("Predicted label ", y_pred)

INFO:tensorflow:Restoring parameters from ./MNIST_MLP/my_model_final_one_hot.ckpt
True label  [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Predicted label  [0]
